In [1]:
import numpy as np
import pandas as pd

from astropy.io import fits
from astropy import units as u
from astropy import constants as const
import matplotlib.pyplot as plt
from scipy.interpolate import InterpolatedUnivariateSpline

from scipy import integrate
# 
# Blackbody imports 
from astropy.modeling.models import BlackBody
from astropy.visualization import quantity_support

In [2]:
pi = np.pi

# Functions for making f_lambda spline from Phoenix model #

In [4]:
# f_lambda function using table
def f_lambda_func(wavelength_array):
    # Use 1e8 instead of 10**8 for non-integers
    return (f_lambda*1e8)*(wavelength_array*1e-8)**2/(const.c.cgs.value)
    
# Spline with f_lambda Phoenix Model
def spline_lambda(f_lambda_table, wavelengthArr, k=1):
    # convert from Angstrom to cm
    wavelengthArr_cm = 1e-8*wavelengthArr
    f_lambda_table_per_cm = f_lambda_table*1e8    
    return InterpolatedUnivariateSpline(wavelengthArr_cm, f_lambda_table_per_cm*wavelengthArr_cm**2/const.c.cgs.value, k=k)

# F_nu function(with f_nu as function of wavelength, transmission, over normal integral)
def passband_flux(flux, transmissionArr, wavelengthsArr):
    int_n = integrate.trapezoid(flux*transmissionArr/wavelengthsArr.cgs.value, x=wavelengthsArr.cgs.value)
    int_d = integrate.trapezoid(transmissionArr/wavelengthsArr.cgs.value, x=wavelengthsArr.cgs.value)
    return int_n/int_d

    


In [5]:
# Returns and compares absolute magnitude against given "actual" magnitude, 
# f_nu (surface flux), R (radius of star), d (distance from observer to star), and F_0 (zero-point flux)

def get_and_compare_mag(f_nu, R, d, F_0, actual_mag=None):
    if actual_mag is None:
        F_M = (f_nu * 4*pi*R**2)/(4*pi*d**2)
        # Absolute magnitude
        M = -2.5 * np.log10(F_M/F_0)
        return M

    else:
        # F_M = (f_nu * 4*pi*R**2)/(4*pi*d**2)
        # Absolute magnitude
        M = -2.5 * np.log10(F_M/F_0)
        print(f"Calculated: {M}, Actual: {actual_mag}")
        return M